In [ ]:
import pandas as pd
import geopandas as gpd
import maup
import numpy as np

# Current Working Directory
working_directory = "/Users/stanleymui/Downloads/CSE 416 Preprocessing Data/"

In [ ]:
# List of Functions
# Function to get the column where cumulative sum equals or exceeds half of TOT_HOUSE21

income_variables = ['TOT_HOUS21','LESS_10K21', '10K_15K21', '15K_20K21', '20K_25K21', '25K_30K21',
                    '30K_35K21','35K_40K21', '40K_45K21', '45K_50K21', '50K_60K21',
                    '60K_75K21', '75K_100K21','100_125K21', '125_150K21', '150_200K21', '200K_MOR21']

def region_category(row):
    if row['Rural'] > row['Urban']:
        return 'Rural'
    elif row['Density'] > 3:
        return 'Urban'
    else:
        return 'Suburban'

def region_category_2(row):
    if row['Rural'] > row['Urban'] and row['Rural'] > row['Suburban']:
        return 'Rural'
    elif row['Suburban'] > row['Rural'] and row['Suburban'] > row['Urban']:
        return 'Suburban'
    else:
        return 'Urban'

In [ ]:
gdf = gpd.read_file(working_directory + "cleaned_merged_output.geojson")
# display(gdf.columns.tolist())

# Demographic and Voting Data
gdf_alabama_demographic_voting = gdf[['GEOID20','Donald J. Trump', 'Joseph R. Biden','TOT_POP22', 'HSP_POP22', 'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22', 'OTH_NHSP22', '2OM_NHSP22', 'geometry']]
gdf_alabama_demographic_voting = gdf_alabama_demographic_voting.rename(columns={'GEOID20': 'SRPREC_KEY','Donald J. Trump': 'PRSDEM01', 'Joseph R. Biden': 
                                                                                'PRSREP01', 'TOT_POP22': 'TOT_POP', 'HSP_POP22': 'POP_HISLAT',
                                                                               'WHT_NHSP22': 'POP_WHT', 'BLK_NHSP22':'POP_BLK', 'AIA_NHSP22':'POP_AINDALK',
                                                                                'ASN_NHSP22':'POP_ASN', 'HPI_NHSP22':'POP_HIPI', 'OTH_NHSP22':'POP_OTH', '2OM_NHSP22':'POP_TWOMOR'})

gdf_alabama_demographic_voting['PRSDEM01'] = gdf_alabama_demographic_voting['PRSDEM01'].astype(int)
gdf_alabama_demographic_voting['PRSREP01'] = gdf_alabama_demographic_voting['PRSREP01'].astype(int)

gdf_alabama_demographic_voting['TOT_VOTES'] = (
    gdf_alabama_demographic_voting['PRSDEM01'] + gdf_alabama_demographic_voting['PRSREP01']
)

gdf_alabama_demographic_voting.insert(3, 'TOT_VOTES', gdf_alabama_demographic_voting.pop('TOT_VOTES'))

gdf_alabama_demographic_voting['PCT_DEM'] = (gdf_alabama_demographic_voting['PRSDEM01'] / gdf_alabama_demographic_voting['TOT_VOTES']) * 100
gdf_alabama_demographic_voting['PCT_REP'] = (gdf_alabama_demographic_voting['PRSREP01'] / gdf_alabama_demographic_voting['TOT_VOTES']) * 100

gdf_alabama_demographic_voting.insert(1, 'PCT_DEM', gdf_alabama_demographic_voting.pop('PCT_DEM'))
gdf_alabama_demographic_voting.insert(2, 'PCT_REP', gdf_alabama_demographic_voting.pop('PCT_REP'))
gdf_alabama_demographic_voting.insert(6, 'geometry', gdf_alabama_demographic_voting.pop('geometry'))

display(gdf_alabama_demographic_voting)

gdf_alabama_demographic_voting["geometry"] = gdf_alabama_demographic_voting.geometry.buffer(0)

# # Check if geometries are valid
# validity_check = gdf_alabama_demographic_voting.is_valid

# # To get a summary of invalid geometries, you can filter them
# invalid_geometries = gdf_alabama_demographic_voting[~validity_check]

# # Display invalid geometries
# print("Invalid geometries:\n", invalid_geometries)

gdf_alabama_block = gpd.read_file(working_directory + "AL_block_2020_new.json")

display(gdf_alabama_block)

In [ ]:
df_alabama_block = pd.read_csv(working_directory + "nhgis0019_ds258_2020_block.csv")

# display(df_alabama_block)

df_alabama_block = df_alabama_block[['GISJOIN', 'AREALAND', 'U7I002', 'U7I003']]
df_alabama_block = df_alabama_block.rename(columns={'U7I002':'Urban', 'U7I003':'Rural'})
gdf_alabama_block = gdf_alabama_block[['GISJOIN', 'geometry']]

gdf_alabama_block = gdf_alabama_block.merge(df_alabama_block, on='GISJOIN')

# display(gdf_alabama_block)

variables = ['AREALAND', 'Urban', 'Rural']

blocks_to_precincts_assignment = maup.assign(gdf_alabama_block, gdf_alabama_demographic_voting)
gdf_alabama_demographic_voting[variables] = gdf_alabama_block[variables].groupby(blocks_to_precincts_assignment).sum()

gdf_alabama_demographic_voting.fillna(0, inplace=True)

gdf_alabama_demographic_voting['Suburban'] = 0.0
gdf_alabama_demographic_voting['Density'] = (gdf_alabama_demographic_voting['TOT_POP'] / gdf_alabama_demographic_voting['AREALAND']) * 1000
gdf_alabama_demographic_voting['Category'] = gdf_alabama_demographic_voting.apply(region_category, axis=1)
gdf_alabama_demographic_voting.loc[gdf_alabama_demographic_voting['Category'] == 'Suburban', 'Suburban'] = gdf_alabama_demographic_voting['Urban']
gdf_alabama_demographic_voting.loc[gdf_alabama_demographic_voting['Category'] == 'Rural', 'Suburban'] = gdf_alabama_demographic_voting['Urban']
gdf_alabama_demographic_voting.loc[gdf_alabama_demographic_voting['Category'] == 'Suburban', 'Urban'] = 0.0
gdf_alabama_demographic_voting.loc[gdf_alabama_demographic_voting['Category'] == 'Rural', 'Urban'] = 0.0

#Congressional District Boundary Data
gdf_congressional_district = gpd.read_file(working_directory + 'al_pl2020_cd.json')

# Assign Precincts To Congressional Districts
precinct_to_cd_assignment = maup.assign(gdf_alabama_demographic_voting, gdf_congressional_district)
gdf_alabama_demographic_voting['CD_ID'] = precinct_to_cd_assignment + 1

display(gdf_alabama_demographic_voting)

In [ ]:
gdf = gpd.read_file(working_directory + "cleaned_merged_output.geojson")
# display(gdf.columns.tolist())

gdf_precinct_income = gdf[['GEOID20', 'geometry', 'TOT_HOUS22', 'LESS_10K22', '10K_15K22', '15K_20K22', '20K_25K22', '25K_30K22',
                           '30K_35K22','35K_40K22','40K_45K22', '45K_50K22','50K_60K22','60K_75K22',
                           '75K_100K22','100_125K22','125_150K22','150_200K22','200K_MOR22', 'MEDN_INC22']]


gdf_precinct_income = gdf_precinct_income.rename(columns={'GEOID20': 'SRPREC_KEY', 'TOT_HOUS22': 'TOT_HOUS21', 'LESS_10K22':'LESS_10K21', '10K_15K22':'10K_15K21', '15K_20K22':'15K_20K21', '20K_25K22':'20K_25K21', 
                           '25K_30K22':'25K_30K21', '30K_35K22':'30K_35K21','35K_40K22':'35K_40K21','40K_45K22':'40K_45K21', '45K_50K22':'45K_50K21','50K_60K22':'50K_60K21','60K_75K22':'60K_75K21',
                           '75K_100K22':'75K_100K21','100_125K22':'100_125K21','125_150K22':'125_150K21','150_200K22':'150_200K21','200K_MOR22':'200K_MOR21', 'MEDN_INC22':'MEDN_INC21'})


# Determine Poverty Level and Percentage
gdf_precinct_income['POVERTY'] = gdf_precinct_income['LESS_10K21'] + gdf_precinct_income['10K_15K21'] + gdf_precinct_income['15K_20K21'] 
+ gdf_precinct_income['20K_25K21'] + gdf_precinct_income['25K_30K21'] + gdf_precinct_income['30K_35K21'] + gdf_precinct_income['35K_40K21']

gdf_precinct_income['POVERTY_PCT'] = np.where(gdf_precinct_income['TOT_HOUS21'] == 0.0, 0.0, (gdf_precinct_income['POVERTY'] / gdf_precinct_income['TOT_HOUS21']) * 100)


#Assign Congresstional District
gdf_precinct_income['CD_ID'] = precinct_to_cd_assignment + 1

gdf_precinct_income["geometry"] = gdf_precinct_income.geometry.buffer(0)

gdf_precinct_income.fillna(0, inplace=True)

display(gdf_precinct_income)

In [ ]:
gdf_alabama_demographic_voting.to_crs(inplace=True, crs="EPSG:4326")
gdf_precinct_income.to_crs(inplace=True, crs="EPSG:4326")

merge_keys = ['SRPREC_KEY', 'CD_ID', 'geometry']

# Merge the GeoDataFrames, keeping all rows and adding unique columns from each
merged_gdf = gdf_alabama_demographic_voting.merge(gdf_precinct_income, on=merge_keys, how='outer')

# Display the merged GeoDataFrame
display(merged_gdf)

merged_gdf.to_file(working_directory + "alabama_precinct_merged.geojson", driver="GeoJSON")

In [ ]:
# read the congressional district file for voting and demographic

gdf_congressional_district = gpd.read_file(working_directory + 'al_pl2020_cd.json')
gdf_congressional_district = gdf_congressional_district[['CD116FP', 'geometry']]
gdf_congressional_district['CD116FP'] = gdf_congressional_district['CD116FP'].astype(int)
gdf_congressional_district = gdf_congressional_district.rename(columns={'CD116FP':'ID'})

voting_and_demographic = ['PCT_DEM', 'PCT_REP', 'PRSDEM01', 'PRSREP01', 'TOT_VOTES', 
                          'TOT_POP', 'POP_HISLAT', 'POP_WHT', 'POP_BLK', 'POP_AINDALK', 'POP_ASN', 'POP_HIPI', 'POP_OTH', 'POP_TWOMOR',
                          'AREALAND', 'Urban', 'Rural', 'Suburban']

# assign each precinct to the congressional district

precinct_to_cd_assignment = maup.assign(gdf_alabama_demographic_voting, gdf_congressional_district)
gdf_congressional_district[voting_and_demographic] = gdf_alabama_demographic_voting[voting_and_demographic].groupby(precinct_to_cd_assignment).sum()

gdf_congressional_district.fillna(0, inplace=True)

# Calculate the percentage of Democratic votes
gdf_congressional_district['PCT_DEM'] = (gdf_congressional_district['PRSDEM01'] / gdf_congressional_district['TOT_VOTES']) * 100
# Calculate the percentage of Republican votes
gdf_congressional_district['PCT_REP'] = (gdf_congressional_district['PRSREP01'] / gdf_congressional_district['TOT_VOTES']) * 100

# Calculate the Category Region Type
gdf_congressional_district['Category'] = gdf_congressional_district.apply(region_category_2, axis=1)

# read the congressional district file for income

gdf_congressional_district_income = gpd.read_file(working_directory + 'al_pl2020_cd.json')
gdf_congressional_district_income = gdf_congressional_district_income[['CD116FP', 'geometry']]
gdf_congressional_district_income['CD116FP'] = gdf_congressional_district_income['CD116FP'].astype(int)
gdf_congressional_district_income= gdf_congressional_district_income.rename(columns={'CD116FP':'ID'})

precinct_to_cd_assignment = maup.assign(gdf_precinct_income, gdf_congressional_district)
gdf_congressional_district_income[income_variables] = gdf_precinct_income[income_variables].groupby(precinct_to_cd_assignment).sum()
gdf_congressional_district_income['MEDN_INC21'] = gdf_precinct_income['MEDN_INC21'].groupby(precinct_to_cd_assignment).mean()

gdf_congressional_district_income.fillna(0, inplace=True)

# Apply the function to each row and store the result in a new column
gdf_congressional_district_income['POVERTY'] = gdf_congressional_district_income['LESS_10K21'] + gdf_congressional_district_income['10K_15K21'] + gdf_congressional_district_income['15K_20K21'] 
+ gdf_congressional_district_income['20K_25K21'] + gdf_congressional_district_income['25K_30K21'] + gdf_congressional_district_income['30K_35K21'] + gdf_congressional_district_income['35K_40K21']

gdf_congressional_district_income['POVERTY_PCT'] = np.where(gdf_congressional_district_income['TOT_HOUS21'] == 0.0, 0.0, (gdf_congressional_district_income['POVERTY'] / gdf_congressional_district_income['TOT_HOUS21']) * 100)


gdf_congressional_district.to_crs(inplace=True, crs="EPSG:4326")
gdf_congressional_district_income.to_crs(inplace=True, crs="EPSG:4326")

# # state district voting and demographic data
# display(gdf_congressional_district)
# # state district income data
# display(gdf_congressional_district_income)

merge_keys = ['ID', 'geometry']

# Merge the GeoDataFrames, keeping all rows and adding unique columns from each
merged_gdf = gdf_congressional_district.merge(gdf_congressional_district_income, on=merge_keys, how='outer')

# Display the merged GeoDataFrame
display(merged_gdf)

# # Check if geometries are valid
# validity_check = gdf_alabama_demographic_voting.is_valid

# # To get a summary of invalid geometries, you can filter them
# invalid_geometries = gdf_alabama_demographic_voting[~validity_check]

# # Display invalid geometries
# print("Invalid geometries:\n", invalid_geometries)

merged_gdf.to_file(working_directory + "alabama_congressional_district_merged.geojson", driver="GeoJSON")

# gdf_congressional_district.to_file(working_directory + "congressional_district_voting_demographic.geojson", driver="GeoJSON")
# gdf_congressional_district_income.to_file(working_directory + "congressional_district_income.geojson", driver="GeoJSON")

In [ ]:
# State Data
# read the state file for voting and demographic

gdf_state = gpd.read_file(working_directory + 'al_st_2020_bound.json')
gdf_state = gdf_state[['NAME20', 'geometry']]
gdf_state = gdf_state.rename(columns={'NAME20':'NAME'})


voting_and_demographic = ['PCT_DEM', 'PCT_REP', 'PRSDEM01', 'PRSREP01', 'TOT_VOTES', 
                          'TOT_POP', 'POP_HISLAT', 'POP_WHT', 'POP_BLK', 'POP_AINDALK', 'POP_ASN', 'POP_HIPI', 'POP_OTH', 'POP_TWOMOR',
                          'AREALAND', 'Urban', 'Rural', 'Suburban']

# assign each precinct to the state

precinct_to_state_assignment = maup.assign(gdf_alabama_demographic_voting, gdf_state)
gdf_state[voting_and_demographic] = gdf_alabama_demographic_voting[voting_and_demographic].groupby(precinct_to_state_assignment).sum()

gdf_state.fillna(0, inplace=True)

# Calculate the percentage of Democratic votes
gdf_state['PCT_DEM'] = (gdf_state['PRSDEM01'] / gdf_state['TOT_VOTES']) * 100
# Calculate the percentage of Republican votes
gdf_state['PCT_REP'] = (gdf_state['PRSREP01'] / gdf_state['TOT_VOTES']) * 100

# Calculate the Category Region Type
gdf_state['Category'] = gdf_state.apply(region_category_2, axis=1)

# read the congressional district file for income

gdf_state_income = gpd.read_file(working_directory + 'CA_State_New.json')
gdf_state_income = gdf_state_income[['NAME', 'geometry']]
gdf_state_income.to_crs(inplace=True, crs="EPSG:3857")

gdf_state_income = gpd.read_file(working_directory + 'al_st_2020_bound.json')
gdf_state_income = gdf_state_income[['NAME20', 'geometry']]
gdf_state_income = gdf_state_income.rename(columns={'NAME20':'NAME'})

precinct_to_state_assignment = maup.assign(gdf_precinct_income, gdf_state_income)
gdf_state_income[income_variables] = gdf_precinct_income[income_variables].groupby(precinct_to_state_assignment).sum()
gdf_state_income['MEDN_INC21'] = gdf_precinct_income['MEDN_INC21'].groupby(precinct_to_state_assignment).mean()

gdf_state_income.fillna(0, inplace=True)
gdf_state_income['POVERTY'] = gdf_state_income['LESS_10K21'] + gdf_state_income['10K_15K21'] + gdf_state_income['15K_20K21'] 
+ gdf_state_income['20K_25K21'] + gdf_state_income['25K_30K21'] + gdf_state_income['30K_35K21'] + gdf_state_income['35K_40K21']

gdf_state_income['POVERTY_PCT'] = np.where(gdf_state_income['TOT_HOUS21'] == 0.0, 0.0, (gdf_state_income['POVERTY'] / gdf_state_income['TOT_HOUS21']) * 100)

# # state voting and demographic data
# display(gdf_state)
# # state income data
# display(gdf_state_income)

merge_keys = ['NAME', 'geometry']

# Merge the GeoDataFrames, keeping all rows and adding unique columns from each
merged_gdf = gdf_state.merge(gdf_state_income, on=merge_keys, how='outer')

# Display the merged GeoDataFrame
display(merged_gdf)

# making 1 as california
# print(merged_gdf.columns.tolist())

merged_gdf.to_file(working_directory + "alabama_state_merged.geojson", driver="GeoJSON")

# gdf_state.to_file(working_directory + "state_voting_demographic.geojson", driver="GeoJSON")
# gdf_state_income.to_file(working_directory + "state_income.geojson", driver="GeoJSON")